In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_pickle('2015_fires.pkl')

In [3]:
natural = []

for idx, row in df.iterrows():
    if row[0] == 'Lightning':
        natural.append(1)
    else:
        natural.append(0)

df['natural_cause'] = natural

In [4]:
df.head()

,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE,STATE,DISCOVERY_DATE,FIRE_SIZE,DATE,MONTH,YEAR,DAY_OF_WEEK,MTHYR,tmax_5,tmin_5,prcp_5,snow_5,last_7,last_14,last_28,natural_cause
0,Campfire,60.910278,-149.458611,AK,2457162.5,0.10,2015-05-20,5,2015,Wednesday,2015-5,15.68,3.20,0.0,0.0,0.000000,165.071429,98.750000,0
1,Campfire,60.507778,-149.811111,AK,2457173.5,0.10,2015-05-31,5,2015,Sunday,2015-5,25.78,5.54,0.0,0.0,0.714286,3.071429,23.500000,0
2,Campfire,60.435000,-149.343333,AK,2457220.5,0.10,2015-07-17,7,2015,Friday,2015-7,15.28,8.52,33.0,0.0,139.857143,77.714286,62.785714,0
3,Miscellaneous,60.399722,-149.643889,AK,2457214.5,0.17,2015-07-11,7,2015,Saturday,2015-7,18.80,7.96,13.2,0.0,10.571429,25.285714,13.928571,0
4,Campfire,60.814444,-149.536389,AK,2457164.5,61.60,2015-05-22,5,2015,Friday,2015-5,20.86,3.24,0.0,0.0,0.000000,72.500000,75.107143,0


In [5]:
# df = df[df['FIRE_SIZE'] > 100]

In [6]:
print(df.corr(method='pearson'))

                LATITUDE  LONGITUDE  DISCOVERY_DATE  FIRE_SIZE     MONTH  \
LATITUDE        1.000000  -0.069004        0.053674   0.180194  0.047325   
LONGITUDE      -0.069004   1.000000       -0.155465  -0.181578 -0.108462   
DISCOVERY_DATE  0.053674  -0.155465        1.000000   0.051217  0.972592   
FIRE_SIZE       0.180194  -0.181578        0.051217   1.000000  0.029954   
MONTH           0.047325  -0.108462        0.972592   0.029954  1.000000   
YEAR                 NaN        NaN             NaN        NaN       NaN   
tmax_5          0.312054  -0.174694        0.257351   0.246268  0.183291   
tmin_5          0.228816  -0.335130        0.394323   0.253865  0.321011   
prcp_5         -0.077918   0.124300        0.116672  -0.030498  0.112837   
snow_5         -0.026204   0.011576       -0.082855  -0.011229 -0.072758   
last_7         -0.282977   0.301591        0.179484  -0.105696  0.201399   
last_14        -0.347245   0.337251        0.165891  -0.091890  0.181721   
last_28     

In [7]:
feat_df = pd.concat([df.iloc[:,5], df.iloc[:,7], df.iloc[:,11:]], axis=1)

print(feat_df.corr(method='pearson'))

               FIRE_SIZE     MONTH    tmax_5    tmin_5    prcp_5    snow_5  \
FIRE_SIZE       1.000000  0.029954  0.246268  0.253865 -0.030498 -0.011229   
MONTH           0.029954  1.000000  0.183291  0.321011  0.112837 -0.072758   
tmax_5          0.246268  0.183291  1.000000  0.820729 -0.010539 -0.087716   
tmin_5          0.253865  0.321011  0.820729  1.000000  0.090257 -0.065563   
prcp_5         -0.030498  0.112837 -0.010539  0.090257  1.000000  0.048185   
snow_5         -0.011229 -0.072758 -0.087716 -0.065563  0.048185  1.000000   
last_7         -0.105696  0.201399 -0.176446 -0.056520  0.354545 -0.003566   
last_14        -0.091890  0.181721 -0.137292 -0.046497  0.258617 -0.017729   
last_28        -0.111576  0.135702 -0.178191 -0.070914  0.253821 -0.025369   
natural_cause   0.245798  0.250559  0.572641  0.624796 -0.005106 -0.043684   

                 last_7   last_14   last_28  natural_cause  
FIRE_SIZE     -0.105696 -0.091890 -0.111576       0.245798  
MONTH          0.20

In [8]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesRegressor

In [27]:
X = pd.concat([df.iloc[:,7], df.iloc[:,11:]], axis=1)
y = df.iloc[:,5]              

In [28]:
etr = ExtraTreesRegressor(random_state = 42)
rfe = RFE(etr, 1)
fit = rfe.fit(X, y)

In [29]:
colnames = X.columns
colranks = fit.ranking_

for i, rank in enumerate(colranks):
    colname = colnames[rank-1]
    print('Rank {}: '.format(i+1) + colname + ', column {}'.format(rank-1))

Rank 1: last_28, column 7
Rank 2: tmax_5, column 1
Rank 3: MONTH, column 0
Rank 4: last_14, column 6
Rank 5: natural_cause, column 8
Rank 6: snow_5, column 4
Rank 7: prcp_5, column 3
Rank 8: tmin_5, column 2
Rank 9: last_7, column 5


#### due to correlations etc etc...

#### initial max features can be 4: last_28, tmax_5, natural_cause, prcp_5

In [30]:
X = pd.concat([X.iloc[:,1], X.iloc[:,3], X.iloc[:,7], X.iloc[:,8]])
X = X.values.reshape(-1,4)

In [17]:
from sklearn.model_selection import StratifiedKFold

In [18]:
skf = StratifiedKFold(n_splits = 10, random_state = 42)

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.metrics import mean_squared_error

In [20]:
def in_sample_error(model, X_train, y_train):
    predict = model.predict(X_train)
    mse = mean_squared_error(y_train, predict)
    return np.sqrt(mse)

def out_sample_error(model, X_test, y_test):
    predict = model.predict(X_test)
    mse = mean_squared_error(y_test, predict)
    return np.sqrt(mse)

def errors(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    training_error = in_sample_error(model, X_train, y_train)
    testing_error = out_sample_error(model, X_test, y_test)
    return training_error, testing_error

In [ ]:
models = 




In [35]:
for model in models:
    
    model_train_error = []
    model_test_error = []

    for train_index, test_index in skf.split(X, y):
    
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]
    
        train_error, test_error = errors(model, X_train, y_train, X_test, y_test)

<generator object _BaseKFold.split at 0x7f19681f9e08>

In [31]:
for train_index, test_index in skf.split(X, y):

    X_train, y_train = X[train_index], y[train_index]
    X_test, y_test = X[test_index], y[test_index]
    
    print(X_train)

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.